In [9]:
import sys
sys.path.append('../')

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from scripts.utils import set_mpl, data_path
from scripts.cross_match_scripts import cross_match_data_frames,  add_separation_columns, desi_reliable_magnitudes
set_mpl()

%matplotlib inline
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy import coordinates
import skymapper as skm
import healpy as hp


matplotlib settings set


In [11]:
def find_secure_ctps(desi_x, x_prefix = 'csc'):

    desi_x = desi_x.copy()
    x_name = f'{x_prefix}_name'
    x_ra_name = f'{x_prefix}_ra'
    x_dec_name = f'{x_prefix}_dec'
    x_err_name = f'{x_prefix}_r_98'
    x_false_name = f'{x_prefix}_r_false'


    #remove DESI LIS duplicates
    desi_x = desi_x.query('brick_primary == True')
    print('desi reliable catalog:', len(desi_x))
    desi_x = desi_x.sort_values(by=[x_name, x_ra_name, x_dec_name, 'flux_g'], ascending=[True, True, True, False])
    desi_x = desi_x.drop_duplicates(subset=[x_name, x_ra_name, x_dec_name,  'ra', 'dec'], keep='first')
    print('desi reliable catalog after non-primary brick and duplicates removal:', len(desi_x))

    #calculate r_false
    rmin = 10 #inner R of annulus in arcsec
    rmax = 30 # outer R of annulus in arcsec
    num_src =  desi_x.groupby(by = [x_name, pd.cut(desi_x.dist_arcsec, [rmin, rmax]) ]).size().unstack()
    src_dens = num_src/annuli_area_deg2(rmin, rmax)
    r_falses = r_false(src_dens)
    r_falses.columns = [x_prefix+'_r_false']
    desi_x = desi_x.merge(r_falses, left_on = x_name, right_index = True)
    desi_x.sort_values(by=[x_name, 'dist_arcsec'], inplace=True)

    #find closest and second closest distances
    gr = desi_x.groupby(x_name)['dist_arcsec']
    desi_x['second_closest_dist'] = gr.transform(lambda x: x.nsmallest(2).max())
    desi_x['closest_dist'] = gr.transform(min)


    #counterparts filter: closest desi objects, but only if they are closer than r_false and r_98. In addition, we demand that there are no other objects closer than r_false
    ctps = desi_x.copy()
    ctps = ctps.query(f"(dist_arcsec==closest_dist) & (closest_dist<{x_false_name} & closest_dist<{x_err_name}) & (second_closest_dist>{x_false_name})")
    print('delete ', ctps.duplicated(subset='desi_id').sum(), ' duplicated desi_id')
    ctps = ctps.drop_duplicates(subset='desi_id')
    print('total number of ctps:', len(ctps))
    ctps['is_counterpart'] = True


    #field sources filter: all desi objects within 10-30 arcsec which are linked to only ONE X-ray source
    field = desi_x.copy()
    field = field.query(f"dist_arcsec>={rmin} & dist_arcsec<={rmax} ")
    print('number of possible field sources: ', len(field))

    tmp_df = field.groupby(by = 'desi_id', ).agg({x_name: 'count'})
    clear_field_ids = tmp_df.index[tmp_df[x_name]==1]
    field = field[field.desi_id.isin(clear_field_ids)==True]
    field['is_counterpart'] = False

    print(f'Final number of secure field sources: ', len(field))
    
    return pd.concat([ctps, field])



def annuli_area_deg2(r_in_arcsec, r_out_arcsec):
    r_in_deg = r_in_arcsec/3600
    r_out_deg = r_out_arcsec/3600
    area = np.pi*(r_out_deg**2 - r_in_deg**2)
    return area

def r_false(desi_rho_deg2, thresh = 0.03):
    ''' Belvedersky+ 2022 '''
    desi_rho_arcsec2 = desi_rho_deg2/(3600**2)
    return np.sqrt(-np.log(1-thresh)/(np.pi*desi_rho_arcsec2))


# load CSC data (all sky, matched with DESI)

the data was obtained with notebooks from `all_sky_crossmatch` repository, see  `2_all-sky-training/0_catalog-download.ipynb` file.

In [12]:
csc = pd.read_pickle(data_path+'csc_all_sky.pkl')
csc = csc.add_prefix('csc_')

csc_desi = pd.read_pickle(data_path+'csc_all_sky_desi_matched.gz_pkl', compression='gzip')
csc_desi.columns = [x.replace('desi_','')  if 'desi' in x else x for x in csc_desi.columns]
csc_desi.rename(columns={'id':'desi_id'}, inplace=True)

cols = [x for x in csc.columns if ('ra'  not in x )&('dec' not in x)  ] #ra or decs are by 1e-5 per cent different in two dataframes in some rows
csc_desi = pd.merge(csc_desi, csc[cols], on=['csc_name'], how='left')
csc_desi = desi_reliable_magnitudes(csc_desi, s_n_threshold=4) #should be 4 for LH paper, also LH coords should be excluded


ra_min = 154.763934
ra_max = 167.084550
dec_min = 54.124219
dec_max = 61.236185
in_lh_mask = csc_desi.eval('(csc_ra >= @ra_min) & (csc_ra <= @ra_max) & (csc_dec >= @dec_min) & (csc_dec <= @dec_max)')
csc_desi = csc_desi[~in_lh_mask]


/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:405: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:405: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:405: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:405: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:405: RuntimeWarning: invalid value encountered in log10
  result = getattr

In [13]:
print(csc_desi.csc_name.nunique())
print(csc_desi.desi_id.nunique())


157958
3142317


In [5]:
csc_training = find_secure_ctps(csc_desi, x_prefix = 'csc')


desi reliable catalog: 3726091
desi reliable catalog after non-primary brick and duplicates removal: 3726084
delete  92  duplicated desi_id
total number of ctps: 80056
number of possible field sources:  3187061
Final number of secure field sources:  2411902


In [6]:
csc_training.groupby('csc_secure')['is_counterpart'].value_counts()

csc_secure  is_counterpart
False       False              276734
            True                 8063
True        False             2135168
            True                71993
Name: is_counterpart, dtype: int64

----

In [7]:
#save cat to pickle
csc_training_secure = csc_training[csc_training.csc_secure==True]
csc_training_secure.to_pickle(data_path+'csc_allsky_desi_r30_gaia_dered_training.gz_pkl', compression='gzip')

In [8]:
csc_training_secure

,csc_name,csc_ra,csc_dec,release,objid,brickid,ra,dec,brick_primary,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,flux_ivar_g,flux_ivar_r,flux_ivar_z,flux_ivar_w1,flux_ivar_w2,flux_ivar_w3,flux_ivar_w4,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,dered_mag_w3,dered_mag_w4,snr_g,snr_r,snr_z,snr_w1,snr_w2,snr_w3,snr_w4,parallax,parallax_ivar,pmra,pmra_ivar,pmdec,pmdec_ivar,dist_arcsec,desi_id,csc_err_ellipse_r0,csc_err_ellipse_r1,csc_err_ellipse_ang,csc_significance,csc_likelihood,csc_likelihood_class,csc_conf_flag,csc_dither_warning_flag,csc_extent_flag,csc_pileup_flag,csc_sat_src_flag,csc_streak_src_flag,csc_var_flag,csc_flux_aper_s,csc_flux_aper_lolim_s,csc_flux_aper_hilim_s,csc_flux_aper_m,csc_flux_aper_lolim_m,csc_flux_aper_hilim_m,csc_r_98,csc_flux_05_2,csc_flux_05_2_err,csc_secure,all_mag_g,rel_mag_g,rel_dered_mag_g,rel_flux_corr_g,all_mag_r,rel_mag_r,rel_dered_mag_r,all_mag_z,rel_mag_z,rel_dered_mag_z,all_mag_w1,rel_mag_w1,rel_dered_mag_w1,vega_mag_w1,all_mag_w2,rel_mag_w2,rel_dered_mag_w2,vega_mag_w2,all_mag_w3,rel_mag_w3,rel_dered_mag_w3,vega_mag_w3,all_mag_w4,rel_mag_w4,rel_dered_mag_w4,vega_mag_w4,rel_flux_corr_r,rel_flux_corr_z,rel_dered_g_r,rel_dered_g_z,rel_dered_r_z,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4,csc_r_false,second_closest_dist,closest_dist,is_counterpart
158,2CXO J000001.4+004822,0.006048,0.806176,9010,236,334688,0.006153,0.806308,1,0.308400,0.731828,1.758477,5.784090,5.948933,72.709250,-3279.48140,1682.19000,948.46300,129.734910,2.735729,0.612379,0.000688,0.000010,23.690054,22.780264,21.854317,20.589422,20.560839,17.845373,NaN,12.648855,22.538210,20.029266,9.566907,4.655316,1.906998,-10.234280,0.0,0.0,0.0,0.0,0.0,0.0,0.607102,9010_334688_236,3.726590,3.135535,46.707312,2.971429,17.027296,TRUE,False,False,False,False,False,False,False,1.384965e-15,3.462414e-16,2.423690e-15,1.626135e-15,8.905024e-16,2.361767e-15,5.565419,3.011100e-15,1.800057e-15,True,23.777214,23.777214,23.690054,1.925701e-15,22.838977,22.838977,22.780264,21.887158,21.887158,21.854317,20.594412,20.594412,20.589422,17.895412,20.563902,20.563902,20.560839,17.224902,17.846026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.700685e-15,3.010245e-15,0.909790,1.835737,0.925947,1.264895,2.219425,0.028583,NaN,NaN,NaN,0.916654,3.528270,0.607102,True
205,2CXO J000001.7+003824,0.007351,0.640170,9010,289,334688,0.007209,0.640172,1,0.446684,0.791833,0.833604,2.084917,-1.714462,-19.942390,-1472.02190,3490.00300,2314.69970,349.076630,3.086889,0.657554,0.000768,0.000011,23.288145,22.694910,22.664875,21.697306,NaN,NaN,NaN,26.388424,38.096140,15.574721,3.663103,-1.390252,-0.552538,-4.863450,0.0,0.0,0.0,0.0,0.0,0.0,0.510857,9010_334688_289,1.093394,0.871849,104.849964,5.485207,110.292372,TRUE,False,False,False,False,False,False,False,2.565676e-15,1.782927e-15,3.348424e-15,2.359721e-15,1.587449e-15,3.131993e-15,1.598058,4.925396e-15,1.555056e-15,True,23.374999,23.374999,23.288145,2.788383e-15,22.753416,22.753416,22.694910,22.697601,22.697601,22.664875,21.702278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.921564e-15,1.426851e-15,0.593235,0.623270,0.030035,NaN,NaN,NaN,NaN,NaN,NaN,0.744180,4.253834,0.510857,True
247,2CXO J000001.7-500850,0.007301,-50.147369,9010,148,76434,0.007012,-50.147418,1,0.284839,0.432803,0.867132,6.043220,7.337446,32.474842,-380.95505,3240.27600,2658.35080,174.380580,4.183603,1.071669,0.001249,0.000015,23.822926,23.381940,22.639498,20.544506,20.334711,18.720829,NaN,16.214005,22.314981,11.450764,12.360716,7.595829,1.147492,-1.469903,0.0,0.0,0.0,0.0,0.0,0.0,0.687582,9010_76434_148,0.743637,0.740034,136.172581,5.772718,368.270366,TRUE,False,False,False,False,False,False,True,1.081031e-15,7.265947e-16,1.417746e-15,9.389929e-16,6.843508e-16,1.193635e-15,1.198873,2.020024e-15,6.071961e-16,True,23.863501,23.863501,23.822926,1.703884e-15,23.409274,23.409274,23.381940,22.654787,22.654787,22.639498,20.546829,20.546829,20.544506,17.847829,20.336138,20.336138,20.334711,16.9